In [4]:
import FinanceDataReader as fdr
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime

# 종목 코드
single_stock_code = '379800'
multi_stock_codes = ['379800', '308620', '411060']

# 날짜 범위
start_date = '2022-01-01'
end_date = datetime.today().strftime('%Y-%m-%d')

# 데이터 가져오기
single_stock_data = fdr.DataReader(single_stock_code, start_date, end_date).resample('ME').last()
multi_stock_data = {code: fdr.DataReader(code, start_date, end_date)['Close'] for code in multi_stock_codes}
multi_stock_data = pd.DataFrame(multi_stock_data).resample('ME').last().dropna()

# 월별 리밸런싱 수익률 계산 함수
def monthly_rebalance_return(df, weights):
    rebalance_returns = (df.pct_change().shift(-1) * weights).sum(axis=1)
    rebalance_cum_returns = (1 + rebalance_returns).cumprod()
    rebalance_cum_returns.iloc[0] = 1
    return rebalance_cum_returns

# 월별 리밸런싱 데이터 준비
weights = np.array([1/len(multi_stock_codes)] * len(multi_stock_codes))
rebalance_cum_returns = monthly_rebalance_return(multi_stock_data, weights)

# MDD 계산 함수
def calculate_mdd(cum_returns):
    drawdown = cum_returns / cum_returns.cummax() - 1
    mdd = drawdown.cummin()
    return mdd

# 한 종목의 수익률 및 MDD 계산
single_stock_cum_returns = (1 + single_stock_data['Close'].pct_change()).cumprod()
single_stock_mdd = calculate_mdd(single_stock_cum_returns)

# 리밸런싱 포트폴리오 수익률 및 MDD 계산
rebalance_mdd = calculate_mdd(rebalance_cum_returns)

# Plotly 그래프 생성 - 수익률 그래프
fig_returns = go.Figure()
fig_returns.add_trace(go.Scatter(
    x=single_stock_data.index,
    y=single_stock_cum_returns,
    name="379800 Cumulative Returns",
    mode="lines+markers+text",
    text=single_stock_cum_returns.round(2),
    textposition="top center",
    texttemplate="%{text}"
))
fig_returns.add_trace(go.Scatter(
    x=rebalance_cum_returns.index,
    y=rebalance_cum_returns,
    name="Rebalanced Portfolio Cumulative Returns",
    mode="lines+markers+text",
    text=rebalance_cum_returns.round(2),
    textposition="top center",
    texttemplate="%{text}"
))
fig_returns.update_layout(
    title="Monthly Cumulative Returns: Single Stock vs Rebalanced Portfolio",
    xaxis=dict(title="Date"),
    yaxis=dict(title="Cumulative Returns")
)

# Plotly 그래프 생성 - MDD 그래프
fig_mdd = go.Figure()
fig_mdd.add_trace(go.Scatter(
    x=single_stock_data.index,
    y=single_stock_mdd,
    name="379800 MDD",
    mode="lines+markers+text",
    text=single_stock_mdd.round(2),
    textposition="bottom center",
    texttemplate="%{text}"
))
fig_mdd.add_trace(go.Scatter(
    x=rebalance_mdd.index,
    y=rebalance_mdd,
    name="Rebalanced Portfolio MDD",
    mode="lines+markers+text",
    text=rebalance_mdd.round(2),
    textposition="bottom center",
    texttemplate="%{text}"
))
fig_mdd.update_layout(
    title="Monthly MDD: Single Stock vs Rebalanced Portfolio",
    xaxis=dict(title="Date"),
    yaxis=dict(title="MDD")
)

fig_returns.show()
fig_mdd.show()
